In [1]:
import pandas as pd
import re
import json 
import sys

sys.path.append('../utils')

from Album import *
from Artist import *
from NumpyEncoder import * 

from utils import *


In [2]:
df = pd.read_csv('../../data/albums/albums_top5000-final.csv',sep=',')
albums_json = None
with open('../../data/albums/missing_albums_details.json',encoding="UTF-8") as f:
    albums_json = json.load(f)      
df_missing_alb = pd.read_csv('../../data/albums/missing_albums_ratings.csv',sep=',')

In [3]:
df_missing_alb

,id,rank,rating,year
0,14541297,8,4.07,10 October 1969
1,23290832,9,4.10,March 1961
2,549847,11,4.09,February 1971
3,18150907,12,4.05,January 1959
4,5923976,17,4.03,March 1960
...,...,...,...,...
1190,11207632,5017,3.59,1976
1191,532535,5021,3.49,2000
1192,3966253,5023,3.55,1955
1193,1109874,5026,3.86,1996


In [3]:
alb_title = df['title']
alb_id = df['album_id']
art_id = df['artist_id']
art_name = df['artist_name']
roles = df['role']
ratings = df['rating']
ranks = df['rank']
years = df['year'] 
labels = df['label']
artists = list()
current_alb = alb_id[0]
albums = list() 
utils = Utils()
unique_role = set()
for i in range(0,len(alb_id)):
    if current_alb != alb_id[i] :
        album = Album(alb_id[i-1],alb_title[i-1],artists,None,years[i-1],labels[i],None,ranks[i-1],ratings[i-1])
        albums.append(album.to_dict())
        current_alb = alb_id[i]
        artists = list()

    clean_roles = list()
    role = utils.clean_role(roles[i])      
    for r in role.split(",") : 
        if "a&r" not in  r :
            r = r.strip()
            clean_roles.append(r)
            if r not in unique_role :
                unique_role.add(r)

    artist = Artist(art_id[i],art_name[i],clean_roles)
    artists.append(artist.to_dict())

ranks = df_missing_alb['rank']
ratings = df_missing_alb['rating']
years = df_missing_alb['year'] 

i = 0
for tuple in albums_json:
    result = utils.get_albums_info_from_json(tuple)
    if result == None :
        continue
    albums_id,album_title,artists,label=result
    
    album = Album(albums_id,album_title,artists,None,years[i],label,None,ranks[i],ratings[i])
    i+=1
with open("album-final.json", "w",encoding='utf-8') as write_file:
    json.dump(albums, write_file, indent=4,cls=NumpyEncoder)  

In [4]:
len(unique_role)

461

In [5]:
unique_role

{'12 string acoustic guitar',
 '6 string banjo',
 'accompanied',
 'accordion',
 'acoustic bass',
 'acoustic guitar',
 'acoustic piccolo bass',
 'adapted',
 'administrator',
 'advisor',
 'afoxé',
 'agogô',
 'alphorn',
 'alto clarinet',
 'alto flute',
 'alto horn',
 'alto saxophone',
 'alto vocals',
 'angklung',
 'arco bass',
 'arranged',
 'art direction',
 'artwork',
 'assemblage',
 'authoring',
 'autoharp',
 'backing band',
 'backing vocals',
 'baglama',
 'bagpipes',
 'balafon',
 'balalaika',
 'band',
 'bandoneon',
 'banjo',
 'bansuri',
 'baritone guitar',
 'baritone horn',
 'baritone saxophone',
 'baryton',
 'bass',
 'bass clarinet',
 'bass drum',
 'bass flute',
 'bass guitar',
 'bass saxophone',
 'bass trombone',
 'bass trumpet',
 'bass tuba',
 'bass viol',
 'basset horn',
 'bassoon',
 'bata',
 'beats',
 'bell tree',
 'bells',
 'bendir',
 'berimbau',
 'biwa',
 'bodhrán',
 'bombarde',
 'bongos',
 'booking',
 'bouzouki',
 'brass',
 'bugle',
 'c melody saxophone',
 'cabasa',
 'cajón',
 